# 第二章 字符串和文本


## 2.1 使用多个界定符来分割字符串

你需要将一个字符串分割为多个字段，但是分隔符(还有周围的空格)并不是固定的。



In [1]:
line = 'asdf fjdk; afed, fjek,asdf, foo'
import re
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

In [3]:
# 使用括号捕获分组
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

如果你不想保留分割字符串到结果列表中去，但仍然需要使用到括号来分组正则表达式的话， 确保你的分组是非捕获分组，形如 (?:...) 。比如：



In [4]:
re.split(r'(?:,|;|\s)\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

## 2.2 字符串或结尾匹配

你需要通过指定的文本模式去检查字符串的开头或者结尾，比如文件名后缀，URL Scheme等等。



In [5]:
filename = 'spam.txt'
filename.endswith('.txt')

True

In [6]:
url = 'http://www.python.org'
url.startswith('http:')

True

In [3]:
from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http:', 'https:', 'ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()

这个方法必需输入一个元组作为参数。

In [4]:
choices = ['http:', 'ftp:']
url = 'htt[://www.python.org'
url.startswith(choices)

TypeError: startswith first arg must be str or a tuple of str, not list

In [5]:
url.startswith(tuple(choices))

False

## 2.3 用shell通配符匹配字符串

你想使用 Unix Shell 中常用的通配符(比如 `*.py` , `Dat[0-9]*.csv` 等)去匹配文本字符串



In [6]:
from fnmatch import fnmatch, fnmatchcase
fnmatch('foo.txt', '*.txt')

True

In [7]:
fnmatch('foo.txt', '?oo.txt')


True

In [8]:
fnmatch('Dat45.csv', 'Dat[0-9]*')

True

In [9]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

In [10]:
fnmatch('foo.txt', '*.TXT')

False

In [11]:
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]

In [12]:
from fnmatch import fnmatchcase
[addr for addr in addresses if fnmatchcase(addr, '* ST')]


['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']

In [13]:
[addr for addr in addresses if fnmatchcase(addr, '54[0-9][0-9] *CLARK*')]

['5412 N CLARK ST']

## 2.4 字符串匹配和搜索

In [1]:
text = 'yeah, but no, but yeah, but no, but yeah'

text.startswith('yeah')

True

In [2]:
text.endswith('yeah')

True

In [4]:
text.find('foo'), text.find('no')

(-1, 10)

In [5]:
text1 = '11/27/2012'
text2 = 'Nov 27, 2012'

import re

# \d+ means match one or more digits
if re.match(r'\d+/\d+/\d+', text1):
    print('yes')
else:
    print('no')

yes


In [6]:
if re.match(r'\d+/\d+/\d+', text2):
    print('yes')
else:
    print('no')

no


In [7]:
datepart = re.compile(r'\d+/\d+/\d+')
if datepart.match(text1):
    print('yes')
else:
    print('no')

yes


In [9]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
datepart.findall(text)

['11/27/2012', '3/13/2013']

在定义正则式的时候，通常会利用括号去捕获分组。

捕获分组可以使得后面的处理更加简单，因为可以分别将每个组的内容提取出来。比如：



In [10]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
m = datepat.match('11/27/2012')
m.group(0)

'11/27/2012'

In [11]:
m.group(1)

'11'

In [12]:
month, day, year = m.groups()

In [13]:
month, day, year

('11', '27', '2012')

In [14]:
datepat.findall(text)

[('11', '27', '2012'), ('3', '13', '2013')]

In [15]:
for month, day, year in datepat.findall(text):
    print('{}-{}-{}'.format(year, month, day))

2012-11-27
2013-3-13


findall() 方法会搜索文本并以列表形式返回所有的匹配。 如果你想以迭代方式返回匹配，可以使用 finditer() 方法来代替，比如：

In [16]:
for m in datepat.finditer(text):
    print(m.groups())

('11', '27', '2012')
('3', '13', '2013')


## 2.5 字符串搜索和替换



In [18]:
text = 'yeah, but no, but yeah, but no, but yeah'
text.replace('yeah', "oop")

'oop, but no, but oop, but no, but oop'

In [20]:
# 复杂情况，使用re.sub
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
import re
re.sub(r'(\d+)/(\d+)/(\d+)', r'\3-\1-\2', text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

In [21]:
re.sub(r'(?P<month>\d+)/(?P<day>\d+)/(?P<year>\d+)', r'\g<year>-\g<month>-\g<day>', text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

对于更加复杂的替换，可以传递一个替换回调函数来代替;

回调函数的参数是一个`match`对象，也就是`match()`或者`find()`返回的对象。使用`group()`方法来提取特定的匹配部分，回调函数最后会返回替换字符串。

In [22]:
from calendar import month_abbr
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))

datepat.sub(change_date, text)

'Today is 27 Nov 2012. PyCon starts 13 Mar 2013.'

In [29]:
text


'Today is 11/27/2012. PyCon starts 3/13/2013.'

如果除了替换后的结果外，你还想知道有多少替换发生了，可以使用 re.subn() 来代替。比如：



In [ ]:
newtext, n = datepat.subn(r'\3-\1-\2', text)
newtext, n